⚠️ This project is mandatory for certification bloc #1.

![Kayak](https://seekvectorlogo.com/wp-content/uploads/2018/01/kayak-vector-logo.png)

# Plan your trip with Kayak 


## Company's description 📇

<a href='https://www.kayak.com' target='_blank'>Kayak</a> is a travel search engine that helps user plan their next trip at the best price.

The company was founded in 2004 by Steve Hafner & Paul M. English. After a few rounds of fundraising, Kayak was acquired by <a href='https://www.bookingholdings.com/' target='_blank'>Booking Holdings</a> which now holds: 

* <a href='https://booking.com/' target='_blank'>Booking.com</a>
* <a href='https://kayak.com/' target='_blank'>Kayak</a>
* <a href='https://www.priceline.com/' target='_blank'>Priceline</a>
* <a href='https://www.agoda.com/' target='_blank'>Agoda</a>
* <a href='https://Rentalcars.com/' target='_blank'>RentalCars</a>
* <a href='https://www.opentable.com/' target='_blank'>OpenTable</a>

With over \$300 million revenue a year, Kayak operates in almost all countries and all languages to help their users book travels accros the globe. 

## Project 🚧

The marketing team needs help on a new project. After doing some user research, the team discovered that **70% of their users who are planning a trip would like to have more information about the destination they are going to**. 

In addition, user research shows that **people tend to be defiant about the information they are reading if they don't know the brand** which produced the content. 

Therefore, Kayak Marketing Team would like to create an application that will recommend where people should plan their next holidays. The application should be based on real data about:

* Weather 
* Hotels in the area 

The application should then be able to recommend the best destinations and hotels based on the above variables at any given time. 

## Goals 🎯

As the project has just started, your team doesn't have any data that can be used to create this application. Therefore, your job will be to: 

* Scrape data from destinations 
* Get weather data from each destination 
* Get hotels' info about each destination
* Store all the information above in a data lake
* Extract, transform and load cleaned data from your datalake to a data warehouse

## Scope of this project 🖼️

Marketing team wants to focus first on the best cities to travel to in France. According <a href='https://one-week-in.com/35-cities-to-visit-in-france/' target='_blank'>One Week In.com</a> here are the top-35 cities to visit in France: 

```python 
['Mont Saint Michel',
'St Malo',
'Bayeux',
'Le Havre',
'Rouen',
'Paris',
'Amiens',
'Lille',
'Strasbourg',
'Chateau du Haut Koenigsbourg',
'Colmar',
'Eguisheim',
'Besancon',
'Dijon',
'Annecy',
'Grenoble',
'Lyon',
'Gorges du Verdon',
'Bormes les Mimosas',
'Cassis',
'Marseille',
'Aix en Provence',
'Avignon',
'Uzes',
'Nimes',
'Aigues Mortes',
'Saintes Maries de la mer',
'Collioure',
'Carcassonne',
'Ariege',
'Toulouse',
'Montauban',
'Biarritz',
'Bayonne',
'La Rochelle']
```

Your team should focus **only on the above cities for your project**. 


## Helpers 🦮

To help you achieve this project, here are a few tips that should help you

### Get weather data with an API 

*   Use https://nominatim.org/ to get the gps coordinates of all the cities (no subscription required) Documentation : https://nominatim.org/release-docs/develop/api/Search/

*   Use https://openweathermap.org/appid (you have to subscribe to get a free apikey) and https://openweathermap.org/api/one-call-api to get some information about the weather for the 35 cities and put it in a DataFrame

*   Determine the list of cities where the weather will be the nicest within the next 7 days For example, you can use the values of daily.pop and daily.rain to compute the expected volume of rain within the next 7 days... But it's only an example, actually you can have different opinions on a what a nice weather would be like 😎 Maybe the most important criterion for you is the temperature or humidity, so feel free to change the rules !

*   Save all the results in a `.csv` file, you will use it later 😉 You can save all the informations that seem important to you ! Don't forget to save the name of the cities, and also to create a column containing a unique identifier (id) of each city (this is important for what's next in the project)

*   Use plotly to display the best destinations on a map

### Scrape Booking.com 

Since BookingHoldings doesn't have aggregated databases, it will be much faster to scrape data directly from booking.com 

You can scrap as many information asyou want, but we suggest that you get at least:

*   hotel name,
*   Url to its booking.com page,
*   Its coordinates: latitude and longitude
*   Score given by the website users
*   Text description of the hotel


### Create your data lake using S3 

Once you managed to build your dataset, you should store into S3 as a csv file. 

### ETL 

Once you uploaded your data onto S3, it will be better for the next data analysis team to extract clean data directly from a Data Warehouse. Therefore, create a SQL Database using AWS RDS, extract your data from S3 and store it in your newly created DB. 

## Deliverable 📬

To complete this project, your team should deliver:

* A `.csv` file in an S3 bucket containing enriched information about weather and hotels for each french city

* A SQL Database where we should be able to get the same cleaned data from S3 

* Two maps where you should have a Top-5 destinations and a Top-20 hotels in the area. You can use plotly or any other library to do so. It should look something like this: 

![Map](https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/Kayak_best_destination_project.png)

# Let's start

In [1]:
# Import libs:

import pandas as pd
import requests
import numpy as np
import plotly.express as px
import scrapy
from scrapy.crawler import CrawlerProcess
import logging
import os
from bs4 import BeautifulSoup
import time
import boto3


## Get weather data with an API

In [2]:
# Import list of cities

cities = [
    "Mont Saint Michel",
    "St Malo",
    "Bayeux",
    "Le Havre",
    "Rouen",
    "Paris",
    "Amiens",
    "Lille",
    "Strasbourg",
    "Chateau du Haut Koenigsbourg",
    "Colmar",
    "Eguisheim",
    "Besancon",
    "Dijon",
    "Annecy",
    "Grenoble",
    "Lyon",
    "Gorges du Verdon",
    "Bormes les Mimosas",
    "Cassis",
    "Marseille",
    "Aix en Provence",
    "Avignon",
    "Uzes",
    "Nimes",
    "Aigues Mortes",
    "Saintes Maries de la mer",
    "Collioure",
    "Carcassonne",
    "Ariege",
    "Toulouse",
    "Montauban",
    "Biarritz",
    "Bayonne",
    "La Rochelle",
]


In [3]:
# Creating a dataframe from the list of cities
df_cities = pd.DataFrame(data=cities, columns=["city"])

df_cities


,city
0,Mont Saint Michel
1,St Malo
2,Bayeux
3,Le Havre
4,Rouen
5,Paris
6,Amiens
7,Lille
8,Strasbourg
9,Chateau du Haut Koenigsbourg


### Nominatim API : get the gps coordinates of all the cities => https://nominatim.org/release-docs/latest/

In [4]:
# Try a city with blank space in name to see if formatting needed:

params = {"city": "La Rochelle", "country": "France", "format": "json"}

r = requests.get("https://nominatim.openstreetmap.org/search?", params).json()

r


[{'place_id': 281822562,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'relation',
  'osm_id': 117858,
  'boundingbox': ['46.1331804', '46.1908971', '-1.2419231', '-1.111097'],
  'lat': '46.1591126',
  'lon': '-1.1520434',
  'display_name': 'La Rochelle, Charente-Maritime, Nouvelle-Aquitaine, France métropolitaine, 17000, France',
  'class': 'boundary',
  'type': 'administrative',
  'importance': 0.9114837096874572,
  'icon': 'https://nominatim.openstreetmap.org/ui/mapicons/poi_boundary_administrative.p.20.png'},
 {'place_id': 282096286,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'relation',
  'osm_id': 1215878,
  'boundingbox': ['47.7388479', '47.7647325', '5.7060641', '5.7454734'],
  'lat': '47.75173925',
  'lon': '5.7245822865532014',
  'display_name': 'La Rochelle, Vesoul, Haute-Saône, Bourgogne-Franche-Comté, France métropolitaine, 70120, France',
  'class': 'boundary',

In [5]:
# I don't know 'Chateau du Haut Koenigsbourg', check if Nominatim knows it

params = {"city": "Chateau du Haut Koenigsbourg", "country": "France", "format": "json"}

r = requests.get("https://nominatim.openstreetmap.org/search?", params).json()

r


[{'place_id': 49750339,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'node',
  'osm_id': 4245068168,
  'boundingbox': ['48.2494726', '48.2495726', '7.3454423', '7.3455423'],
  'lat': '48.2495226',
  'lon': '7.3454923',
  'display_name': 'Château du Haut-Kœnigsbourg, Orschwiller, Sélestat-Erstein, Bas-Rhin, Grand Est, France métropolitaine, 67600, France',
  'class': 'place',
  'type': 'isolated_dwelling',
  'importance': 0.51}]

In [6]:
# 'Gorges du Verdon' isn't a city but a canyon, check if Nominatim works with or not

params = {"city": "Gorges du Verdon", "country": "France", "format": "json"}

r = requests.get("https://nominatim.openstreetmap.org/search?", params).json()

r

# Doesnt' works... Google search : 'Castellane' will be use for Gorges du Verdon


[]

Ariege is a department and not a city, its prefecture is 'Foix'. We'll use this city

#### Request API

In [7]:
# Creating a copy of df_cities to store the coordinates from Nominatim API

df_gps = df_cities.copy()
lat_list = []
lon_list = []

for i in cities:
    print(f"Request for city: {i}")
    params = {"city": i, "country": "France", "format": "json"}
    # No 'Gorges du Verdon' city, replacing by 'Castellane'
    if i == "Gorges du Verdon":
        i = "Castellane"
        r = requests.get(
            f"https://nominatim.openstreetmap.org/search?city={i}&country=France&format=json"
        ).json()
        lat_list.append(r[0]["lat"])
        lon_list.append(r[0]["lon"])
    # 'Ariege' not a city, using the prefecture instead -> 'Foix'
    elif i == "Ariege":
        i = "Foix"
        r = requests.get(
            f"https://nominatim.openstreetmap.org/search?city={i}&country=France&format=json"
        ).json()
        lat_list.append(r[0]["lat"])
        lon_list.append(r[0]["lon"])
    else:
        r = requests.get(f"https://nominatim.openstreetmap.org/search?", params).json()
        lat_list.append(r[0]["lat"])
        lon_list.append(r[0]["lon"])

# Adding the coordinates to the dataframe
df_gps["lat"] = lat_list
df_gps["lon"] = lon_list


Request for city: Mont Saint Michel
Request for city: St Malo
Request for city: Bayeux
Request for city: Le Havre
Request for city: Rouen
Request for city: Paris
Request for city: Amiens
Request for city: Lille
Request for city: Strasbourg
Request for city: Chateau du Haut Koenigsbourg
Request for city: Colmar
Request for city: Eguisheim
Request for city: Besancon
Request for city: Dijon
Request for city: Annecy
Request for city: Grenoble
Request for city: Lyon
Request for city: Gorges du Verdon
Request for city: Bormes les Mimosas
Request for city: Cassis
Request for city: Marseille
Request for city: Aix en Provence
Request for city: Avignon
Request for city: Uzes
Request for city: Nimes
Request for city: Aigues Mortes
Request for city: Saintes Maries de la mer
Request for city: Collioure
Request for city: Carcassonne
Request for city: Ariege
Request for city: Toulouse
Request for city: Montauban
Request for city: Biarritz
Request for city: Bayonne
Request for city: La Rochelle


In [8]:
df_gps.head()


,city,lat,lon
0,Mont Saint Michel,48.6359541,-1.511459954959514
1,St Malo,48.649518,-2.0260409
2,Bayeux,49.2764624,-0.7024738
3,Le Havre,49.4938975,0.1079732
4,Rouen,49.4404591,1.0939658


In [9]:
df_gps.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   city    35 non-null     object
 1   lat     35 non-null     object
 2   lon     35 non-null     object
dtypes: object(3)
memory usage: 968.0+ bytes


### OpenWeather : get weather of the week => https://openweathermap.org

In [10]:
# try at St Malo
parameters = {
    "lat": 48.649518,
    "lon": -2.0260409,
    "exclude": "current,minutely,hourly",
    "units": "metric",
    "appid": "aa423e6694bf72625fe1fe31544949dc",
    "lang": "fr",
}

r = requests.get(
    "https://api.openweathermap.org/data/2.5/onecall?", params=parameters
).json()

r


{'lat': 48.6495,
 'lon': -2.026,
 'timezone': 'Europe/Paris',
 'timezone_offset': 7200,
 'daily': [{'dt': 1655985600,
   'sunrise': 1655957154,
   'sunset': 1656015274,
   'moonrise': 1655946300,
   'moonset': 1655995620,
   'moon_phase': 0.82,
   'temp': {'day': 18.28,
    'min': 15.31,
    'max': 18.36,
    'night': 15.31,
    'eve': 17.47,
    'morn': 16.67},
   'feels_like': {'day': 18.3, 'night': 15.09, 'eve': 17.31, 'morn': 16.74},
   'pressure': 1010,
   'humidity': 82,
   'dew_point': 15.16,
   'wind_speed': 4.9,
   'wind_deg': 261,
   'wind_gust': 6.56,
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'légère pluie',
     'icon': '10d'}],
   'clouds': 84,
   'pop': 0.76,
   'rain': 3.19,
   'uvi': 7.35},
  {'dt': 1656072000,
   'sunrise': 1656043572,
   'sunset': 1656101680,
   'moonrise': 1656033720,
   'moonset': 1656086280,
   'moon_phase': 0.86,
   'temp': {'day': 19.4,
    'min': 14,
    'max': 20.06,
    'night': 15.47,
    'eve': 17.76,
    'morn': 15

In [11]:
# Weather in 3 days at St Malo
r["daily"][3:]


[{'dt': 1656244800,
  'sunrise': 1656216416,
  'sunset': 1656274484,
  'moonrise': 1656209040,
  'moonset': 1656267300,
  'moon_phase': 0.92,
  'temp': {'day': 18,
   'min': 11.36,
   'max': 18,
   'night': 14.25,
   'eve': 17.17,
   'morn': 12.9},
  'feels_like': {'day': 17.32, 'night': 13.66, 'eve': 16.61, 'morn': 12.2},
  'pressure': 1012,
  'humidity': 56,
  'dew_point': 8.94,
  'wind_speed': 5.4,
  'wind_deg': 217,
  'wind_gust': 7.81,
  'weather': [{'id': 500,
    'main': 'Rain',
    'description': 'légère pluie',
    'icon': '10d'}],
  'clouds': 71,
  'pop': 0.33,
  'rain': 0.19,
  'uvi': 6.69},
 {'dt': 1656331200,
  'sunrise': 1656302841,
  'sunset': 1656360882,
  'moonrise': 1656297120,
  'moonset': 1656357600,
  'moon_phase': 0.95,
  'temp': {'day': 16.82,
   'min': 12.45,
   'max': 16.96,
   'night': 14.6,
   'eve': 16.08,
   'morn': 13.06},
  'feels_like': {'day': 16.12, 'night': 13.89, 'eve': 15.41, 'morn': 12.46},
  'pressure': 1018,
  'humidity': 60,
  'dew_point': 8.71,

In [12]:
day3 = r["daily"][3]  # Weather in 3 days

# desciption of the main weather in 3 days at St Malo
day3["weather"][0]["main"]


'Rain'

In [13]:
df_gps_weather = df_gps.copy(deep=True)
my_api_key = "aa423e6694bf72625fe1fe31544949dc"

temperatures_list = []
rain_list = []
weather_list = []

days = list(range(1, 8))

for i in df_gps_weather.itertuples():
    lat = i.lat
    lon = i.lon

    parameters = {
        "lat": {lat},
        "lon": {lon},
        "exclude": "current,minutely,hourly",
        "units": "metric",
        "appid": "aa423e6694bf72625fe1fe31544949dc",
        "lang": "fr",
    }

    r = requests.get(
        f"https://api.openweathermap.org/data/3.0/onecall?", parameters
    ).json()
    forecast_7days = r["daily"][1:]  # Getting the weather data for the next 7 days
    temperatures = [int(d["feels_like"]["day"]) for d in forecast_7days]
    rain = [int(d["pop"] * 100) for d in forecast_7days]
    weather = [str(d["weather"][0]["main"]) for d in forecast_7days]
    temperatures_list.append(temperatures)
    rain_list.append(rain)
    weather_list.append(weather)

df_gps_weather["jour_+x"] = [days for _ in range(len(df_gps_weather))]
df_gps_weather["temperature_ressentie"] = temperatures_list
df_gps_weather["probabilite_de_pluie"] = rain_list
df_gps_weather["meteo_principale"] = weather_list
# df_weather['rang'] = df_weather['probabilite_de_pluie'].sort_values()
df_gps_weather["score"] = df_gps_weather.apply(
    lambda x: ((np.mean(x["temperature_ressentie"])))
    - (np.mean(x["probabilite_de_pluie"]) / 10),
    axis=1,
).astype(int)


In [14]:
df_gps_weather


,city,lat,lon,jour_+x,temperature_ressentie,probabilite_de_pluie,meteo_principale,score
0,Mont Saint Michel,48.6359541,-1.511459954959514,"[1, 2, 3, 4, 5, 6, 7]","[18, 19, 17, 17, 20, 17, 18]","[85, 98, 23, 0, 0, 100, 75]","[Rain, Rain, Rain, Clouds, Clouds, Rain, Rain]",12
1,St Malo,48.649518,-2.0260409,"[1, 2, 3, 4, 5, 6, 7]","[19, 18, 17, 16, 19, 18, 18]","[99, 88, 15, 9, 0, 100, 65]","[Rain, Clouds, Clouds, Clouds, Clouds, Rain, R...",12
2,Bayeux,49.2764624,-0.7024738,"[1, 2, 3, 4, 5, 6, 7]","[19, 16, 19, 17, 20, 13, 20]","[90, 93, 73, 37, 0, 100, 69]","[Rain, Rain, Rain, Rain, Clouds, Rain, Rain]",11
3,Le Havre,49.4938975,0.1079732,"[1, 2, 3, 4, 5, 6, 7]","[20, 14, 18, 16, 17, 13, 17]","[74, 100, 25, 18, 0, 98, 64]","[Rain, Rain, Rain, Clouds, Clouds, Rain, Rain]",11
4,Rouen,49.4404591,1.0939658,"[1, 2, 3, 4, 5, 6, 7]","[20, 15, 19, 18, 21, 18, 19]","[97, 100, 30, 61, 0, 42, 62]","[Rain, Rain, Clouds, Rain, Clouds, Rain, Rain]",12
5,Paris,48.8588897,2.3200410217200766,"[1, 2, 3, 4, 5, 6, 7]","[23, 17, 19, 19, 22, 24, 19]","[65, 100, 66, 72, 7, 0, 63]","[Rain, Rain, Rain, Rain, Clouds, Clouds, Rain]",15
6,Amiens,49.8941708,2.2956951,"[1, 2, 3, 4, 5, 6, 7]","[22, 16, 19, 20, 21, 21, 19]","[86, 100, 4, 3, 0, 5, 89]","[Rain, Rain, Clouds, Clouds, Clouds, Clouds, R...",15
7,Lille,50.6365654,3.0635282,"[1, 2, 3, 4, 5, 6, 7]","[22, 19, 20, 21, 21, 22, 19]","[98, 100, 8, 0, 0, 81, 70]","[Rain, Rain, Clouds, Clouds, Clear, Rain, Rain]",15
8,Strasbourg,48.584614,7.7507127,"[1, 2, 3, 4, 5, 6, 7]","[20, 25, 22, 20, 20, 26, 19]","[100, 92, 100, 100, 53, 0, 36]","[Rain, Rain, Rain, Rain, Rain, Clouds, Rain]",14
9,Chateau du Haut Koenigsbourg,48.2495226,7.3454923,"[1, 2, 3, 4, 5, 6, 7]","[19, 22, 20, 16, 17, 22, 15]","[100, 100, 87, 100, 67, 0, 24]","[Rain, Rain, Rain, Rain, Rain, Clear, Rain]",11


In [15]:
df_gps_weather.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   city                   35 non-null     object
 1   lat                    35 non-null     object
 2   lon                    35 non-null     object
 3   jour_+x                35 non-null     object
 4   temperature_ressentie  35 non-null     object
 5   probabilite_de_pluie   35 non-null     object
 6   meteo_principale       35 non-null     object
 7   score                  35 non-null     int64 
dtypes: int64(1), object(7)
memory usage: 2.3+ KB


In [16]:
df_gps_weather[["lat", "lon"]] = df_gps_weather[["lat", "lon"]].astype(float)


### List of cities where the weather will be the nicest within the next 7 days

In [17]:
df_gps_weather.sort_values(by=["score"], ascending=False)


,city,lat,lon,jour_+x,temperature_ressentie,probabilite_de_pluie,meteo_principale,score
18,Bormes les Mimosas,43.157217,6.329254,"[1, 2, 3, 4, 5, 6, 7]","[28, 27, 29, 29, 28, 27, 28]","[23, 1, 1, 28, 15, 0, 0]","[Clouds, Clear, Clear, Clouds, Clear, Clear, C...",27
21,Aix en Provence,43.529842,5.447474,"[1, 2, 3, 4, 5, 6, 7]","[27, 28, 29, 28, 26, 29, 29]","[78, 0, 0, 56, 19, 0, 0]","[Rain, Clear, Clear, Clouds, Clear, Clear, Clear]",25
20,Marseille,43.296174,5.369953,"[1, 2, 3, 4, 5, 6, 7]","[25, 26, 28, 28, 26, 27, 28]","[60, 2, 3, 38, 2, 0, 0]","[Rain, Clear, Clouds, Rain, Clear, Clear, Clouds]",25
19,Cassis,43.214036,5.539632,"[1, 2, 3, 4, 5, 6, 7]","[25, 27, 26, 26, 26, 26, 26]","[52, 0, 6, 34, 0, 0, 0]","[Rain, Clear, Clear, Rain, Clear, Clear, Clouds]",24
17,Gorges du Verdon,43.846218,6.513181,"[1, 2, 3, 4, 5, 6, 7]","[25, 26, 28, 27, 27, 27, 28]","[57, 45, 28, 72, 64, 0, 0]","[Rain, Rain, Clear, Rain, Rain, Clear, Clear]",23
27,Collioure,42.525050,3.083155,"[1, 2, 3, 4, 5, 6, 7]","[26, 29, 25, 22, 26, 28, 23]","[28, 45, 31, 37, 0, 0, 5]","[Rain, Rain, Clouds, Clouds, Clear, Clear, Clo...",23
22,Avignon,43.949249,4.805901,"[1, 2, 3, 4, 5, 6, 7]","[29, 30, 31, 23, 25, 30, 27]","[95, 16, 27, 89, 56, 0, 3]","[Rain, Clear, Clear, Rain, Clouds, Clear, Clouds]",23
24,Nimes,43.837425,4.360069,"[1, 2, 3, 4, 5, 6, 7]","[27, 28, 30, 23, 26, 29, 27]","[81, 41, 50, 94, 47, 0, 21]","[Rain, Rain, Rain, Rain, Clear, Clear, Rain]",22
23,Uzes,44.012128,4.419672,"[1, 2, 3, 4, 5, 6, 7]","[27, 28, 30, 22, 26, 28, 25]","[90, 39, 55, 97, 49, 0, 36]","[Rain, Rain, Rain, Rain, Clear, Clear, Rain]",21
25,Aigues Mortes,43.565823,4.191284,"[1, 2, 3, 4, 5, 6, 7]","[25, 26, 26, 21, 24, 26, 25]","[66, 44, 52, 89, 25, 0, 24]","[Rain, Rain, Rain, Rain, Clouds, Clear, Rain]",20


### Save all the results in a `.csv` file

In [18]:
df_gps_weather = df_gps_weather.set_index("city")
df_gps_weather = df_gps_weather.reindex(cities)
df_gps_weather = df_gps_weather.reset_index()
df_gps_weather = df_gps_weather.rename(columns={"index": "id"})

df_gps_weather.to_csv("df_gps_weather.csv", index=False)


### Plotly to display the best destinations on a map

In [19]:
df_plotly = df_gps_weather.apply(
    pd.Series.explode
)  # To obtain a line per day and per city

df_plotly[["jour_+x", "temperature_ressentie", "probabilite_de_pluie"]] = df_plotly[
    ["jour_+x", "temperature_ressentie", "probabilite_de_pluie"]
].astype(int)

fig = px.scatter_mapbox(
    df_plotly,
    lat="lat",
    lon="lon",
    hover_name="city",
    zoom=4,
    hover_data=["meteo_principale", "probabilite_de_pluie", "temperature_ressentie"],
    color="temperature_ressentie",
    color_continuous_scale="thermal",
    mapbox_style="carto-positron",
)
fig.show()


## Scrape Booking.com 

### Get hotels and their URLs

In [20]:
class BookingSpider(scrapy.Spider):
    # Name of my spider
    name = "booking_spider"
    cities = df_gps_weather.city
    # Url to start my spider from
    start_urls = [
        "https://www.booking.com/index.fr.html",
    ]

    # Callback function that will be called when starting my spider
    def parse(self, response):
        for x in cities:
            yield scrapy.FormRequest.from_response(
                response, formdata={"ss": x}, callback=self.after_search
            )

    def after_search(self, response):
        cities = response.url.split("ss=")[-1].split("&")[0]
        booking = response.css(".d4924c9e74")

        for data in booking:

            yield {
                "ville": cities,
                "hotels": data.css("a div.fcab3ed991.a23c043802::text").getall(),
                "liens": data.css("h3.a4225678b2 a::attr(href)").getall(),
            }

        try:
            next_page = response.css("a.paging-next").attrib["href"]
        except KeyError:
            logging.info("No next page. Terminating crawling process.")
        else:
            yield response.follow(next_page, callback=self.after_search)


# Name of the file where the results will be saved
filename = "hotels.json"

# If file already exists, delete before crawling (because Scrapy will concatenate the last and new results otherwise)
if filename in os.listdir():
    os.remove(filename)

# Declare a new CrawlerProcess with some settings
process = CrawlerProcess(
    settings={
        "USER_AGENT": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.112 Safari/535.1",
        "LOG_LEVEL": logging.INFO,
        "FEEDS": {
            filename: {"format": "json"},
        },
        "AUTOTHROTTLE_ENABLED": True,
    }
)

# Start the crawling using the spider you defined above
process.crawl(BookingSpider)
process.start()


2022-06-23 10:42:23 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-06-23 10:42:23 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.2.0, Python 3.9.12 (main, Apr  5 2022, 01:53:17) - [Clang 12.0.0 ], pyOpenSSL 21.0.0 (OpenSSL 1.1.1o  3 May 2022), cryptography 3.4.8, Platform macOS-10.16-x86_64-i386-64bit
2022-06-23 10:42:23 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'LOG_LEVEL': 20,
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.1 (KHTML, '
               'like Gecko) Chrome/13.0.782.112 Safari/535.1'}
2022-06-23 10:42:23 [scrapy.extensions.telnet] INFO: Telnet Password: eed987bb08649896
2022-06-23 10:42:23 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions

### Get hotels' coordinates

In [3]:
# Create a DataFrame from the json file

df = pd.read_json("hotels.json")
df["ville"] = df["ville"].str.replace("+", " ")
df


/var/folders/hr/m4w2vfm12db4g050bkkr7y540000gn/T/ipykernel_6103/2552240652.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["ville"] = df["ville"].str.replace("+", " ")


,ville,hotels,liens
0,Mont Saint Michel,"[Hôtel Vert, Hotel De La Digue, Le Saint Auber...",[https://www.booking.com/hotel/fr/vert.fr.html...
1,St Malo,"[Le Cargo by Cocoonr, Hotel d'Aleth, Le Hauban...",[https://www.booking.com/hotel/fr/le-cargo-sai...
2,Bayeux,"[Le Mogador, Le Castel Guesthouse, ibis budget...",[https://www.booking.com/hotel/fr/le-mogador-b...
3,Le Havre,"[Hilton Garden Inn Le Havre Centre, Hôtel Le M...",[https://www.booking.com/hotel/fr/hilton-garde...
4,Rouen,"[Radisson Blu Hotel, Rouen Centre, Mercure Rou...",[https://www.booking.com/hotel/fr/radisson-blu...
5,Paris,"[MEININGER Hotel Paris Porte de Vincennes, Cit...",[https://www.booking.com/hotel/fr/meininger-pa...
6,Amiens,"[Holiday Inn Express Amiens, an IHG Hotel, Hot...",[https://www.booking.com/hotel/fr/express-by-h...
7,Lille,"[Moxy Lille City, Hotel Lille Europe, Séjours ...",[https://www.booking.com/hotel/fr/moxy-lille-c...
8,Strasbourg,[Comfort Hotel Strasbourg - Montagne Verte & R...,[https://www.booking.com/hotel/fr/comforthotel...
9,Chateau du Haut Koenigsbourg,"[Les Chambres du Haut-Koenigsbourg, Gîte L'Oré...",[https://www.booking.com/hotel/fr/les-chambres...


In [4]:
'''# Reduce number of hotels per city to 20 as requested by the task
for i in range(len(df["ville"])):
    df["hotels"][i] = df["hotels"][i][0:20]
    df["liens"][i] = df["liens"][i][0:20]
'''

'# Reduce number of hotels per city to 20 as requested by the task\nfor i in range(len(df["ville"])):\n    df["hotels"][i] = df["hotels"][i][0:20]\n    df["liens"][i] = df["liens"][i][0:20]\n'

In [6]:
# BeautifulSoup iterates on each URL to get the hotel's score, GPS coordinates and description.
df["lat"] = 0
df["lon"] = 0
df["description"] = "---"
df["score"] = 0.0

navigator = "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.112 Safari/535.1"

for i in range(len(df["liens"])):
    lat_list = []
    lon_list = []
    description_list = []
    score_list = []

    hotel_list = df["liens"][i]

    for i2 in hotel_list:

        # Sometimes BeautifulSoup doesn't manage to gather data. When it fails, it tries again.
        try:
            page = requests.get(i2, headers={"User-Agent": navigator})
            soup = BeautifulSoup(page.text, "html.parser")
        except:
            page = requests.get(i2, headers={"User-Agent": navigator})
            soup = BeautifulSoup(page.text, "html.parser")

        lat_list.append(
            soup.select("p.address.address_clean a")[0]
            .get("data-atlas-latlng")
            .split(",")[0]
        )
        lon_list.append(
            soup.select("p.address.address_clean a")[0]
            .get("data-atlas-latlng")
            .split(",")[1]
        )
        description_list.append(
            soup.select("div#property_description_content")[0].get_text()
        )

        try:
            score_list.append(soup.select("div.b5cd09854e.d10a6220b4")[0].get_text())

        except:
            # 2 hotels over the 700 that I am going to scrap dont have a score yet but I still need one for the visualization. I set it to 1.
            score_list.append("1.0")

        time.sleep(1.4)

    df["lat"][i] = lat_list
    df["lon"][i] = lon_list
    df["description"][i] = description_list
    df["score"][i] = score_list

    print(f"city {df['ville'].iloc[i]} done")


/var/folders/hr/m4w2vfm12db4g050bkkr7y540000gn/T/ipykernel_5721/2031490341.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["lat"][i] = lat_list
/var/folders/hr/m4w2vfm12db4g050bkkr7y540000gn/T/ipykernel_5721/2031490341.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["lon"][i] = lon_list
/var/folders/hr/m4w2vfm12db4g050bkkr7y540000gn/T/ipykernel_5721/2031490341.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["des

city Mont+Saint+Michel done
city St+Malo done
city Bayeux done
city Le+Havre done
city Rouen done
city Paris done
city Amiens done
city Lille done


In [ ]:
df

In [24]:
df = df.set_index("ville")
df = df.reindex(cities)
df = df.reset_index()
df = df.rename(columns={"index": "id"})
df


,ville,hotels,liens,lat,lon,description,score
0,Mont Saint Michel,NaN,NaN,NaN,NaN,NaN,NaN
1,St Malo,NaN,NaN,NaN,NaN,NaN,NaN
2,Bayeux,"[Le Mogador, Le Castel Guesthouse, ibis budget...",[https://www.booking.com/hotel/fr/le-mogador-b...,"[49.27933593, 49.27368314, 49.25424209, 49.272...","[-0.70705610, -0.70322692, -0.64648747, -0.698...",[\nL'établissement Le Mogador est situé à Baye...,"[8,1, 8,6, 8,2, 9,3, 9,6, 8,6, 9,5, 9,8, 8,1, ..."
3,Le Havre,NaN,NaN,NaN,NaN,NaN,NaN
4,Rouen,"[Radisson Blu Hotel, Rouen Centre, Mercure Rou...",[https://www.booking.com/hotel/fr/radisson-blu...,"[49.44644100, 49.44117285, 49.43783419, 49.438...","[1.09412000, 1.09517545, 1.09725282, 1.1092034...",[\n\nVous pouvez bénéficier d'une réduction Ge...,"[8,9, 8,2, 8,4, 7,5, 8,5, 8,2, 8,2, 8,6, 8,6, ..."
5,Paris,"[MEININGER Hotel Paris Porte de Vincennes, Cit...",[https://www.booking.com/hotel/fr/meininger-pa...,"[48.84410500, 48.86636230, 48.85815936, 48.831...","[2.41306300, 2.28569001, 2.37099834, 2.2810444...","[\nDoté d'un bar et d'un salon commun, le MEIN...","[8,1, 8,5, 8,0, 7,7, 7,8, 7,9, 6,8, 7,3, 8,2, ..."
6,Amiens,"[Holiday Inn Express Amiens, an IHG Hotel, Hot...",[https://www.booking.com/hotel/fr/express-by-h...,"[49.89167620, 49.89444519, 49.89422900, 49.897...","[2.30775386, 2.30094239, 2.30577800, 2.3011536...","[\nRénové en avril 2015, le Holiday Inn Expres...","[8,1, 8,3, 8,0, 8,9, 8,1, 8,8, 7,6, 7,3, 9,5, ..."
7,Lille,"[Moxy Lille City, Hotel Lille Europe, Séjours ...",[https://www.booking.com/hotel/fr/moxy-lille-c...,"[50.62783100, 50.63788756, 50.63647023, 50.635...","[3.06359200, 3.07268500, 3.07490923, 3.0697072...","[\nLe Moxy Lille City est situé à Lille, à 700...","[8,6, 8,0, 7,3, 7,7, 9,2, 8,0, 7,8, 8,5, 8,1, ..."
8,Strasbourg,[Comfort Hotel Strasbourg - Montagne Verte & R...,[https://www.booking.com/hotel/fr/comforthotel...,"[48.57269177, 48.59001400, 48.58906194, 48.604...","[7.72871353, 7.73987000, 7.73879863, 7.7044611...","[\nSitué près de l'Ill, le Comfort Hotel Stras...","[8,3, 7,0, 7,1, 8,6, 7,8, 8,3, 9,0, 8,7, 8,1, ..."
9,Chateau du Haut Koenigsbourg,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df["ville"] = df["ville"].str.replace("+", " ")


2022-06-23 11:34:03 [py.warnings] WARNING: /var/folders/hr/m4w2vfm12db4g050bkkr7y540000gn/T/ipykernel_4159/297886527.py:1: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.




In [26]:
df.to_csv("df_gps_hotels.csv", index=False)


### Test of CSV files

In [27]:
test_hotels = pd.read_csv("df_gps_hotels.csv")
test_hotels


,ville,hotels,liens,lat,lon,description,score
0,Mont Saint Michel,NaN,NaN,NaN,NaN,NaN,NaN
1,St Malo,NaN,NaN,NaN,NaN,NaN,NaN
2,Bayeux,"['Le Mogador', 'Le Castel Guesthouse', 'ibis b...",['https://www.booking.com/hotel/fr/le-mogador-...,"['49.27933593', '49.27368314', '49.25424209', ...","['-0.70705610', '-0.70322692', '-0.64648747', ...","[""\nL'établissement Le Mogador est situé à Bay...","['8,1', '8,6', '8,2', '9,3', '9,6', '8,6', '9,..."
3,Le Havre,NaN,NaN,NaN,NaN,NaN,NaN
4,Rouen,"['Radisson Blu Hotel, Rouen Centre', 'Mercure ...",['https://www.booking.com/hotel/fr/radisson-bl...,"['49.44644100', '49.44117285', '49.43783419', ...","['1.09412000', '1.09517545', '1.09725282', '1....","[""\n\nVous pouvez bénéficier d'une réduction G...","['8,9', '8,2', '8,4', '7,5', '8,5', '8,2', '8,..."
5,Paris,"['MEININGER Hotel Paris Porte de Vincennes', '...",['https://www.booking.com/hotel/fr/meininger-p...,"['48.84410500', '48.86636230', '48.85815936', ...","['2.41306300', '2.28569001', '2.37099834', '2....","[""\nDoté d'un bar et d'un salon commun, le MEI...","['8,1', '8,5', '8,0', '7,7', '7,8', '7,9', '6,..."
6,Amiens,"['Holiday Inn Express Amiens, an IHG Hotel', '...",['https://www.booking.com/hotel/fr/express-by-...,"['49.89167620', '49.89444519', '49.89422900', ...","['2.30775386', '2.30094239', '2.30577800', '2....","['\nRénové en avril 2015, le Holiday Inn Expre...","['8,1', '8,3', '8,0', '8,9', '8,1', '8,8', '7,..."
7,Lille,"['Moxy Lille City', 'Hotel Lille Europe', 'Séj...",['https://www.booking.com/hotel/fr/moxy-lille-...,"['50.62783100', '50.63788756', '50.63647023', ...","['3.06359200', '3.07268500', '3.07490923', '3....","[""\nLe Moxy Lille City est situé à Lille, à 70...","['8,6', '8,0', '7,3', '7,7', '9,2', '8,0', '7,..."
8,Strasbourg,['Comfort Hotel Strasbourg - Montagne Verte & ...,['https://www.booking.com/hotel/fr/comforthote...,"['48.57269177', '48.59001400', '48.58906194', ...","['7.72871353', '7.73987000', '7.73879863', '7....","[""\nSitué près de l'Ill, le Comfort Hotel Stra...","['8,3', '7,0', '7,1', '8,6', '7,8', '8,3', '9,..."
9,Chateau du Haut Koenigsbourg,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
test_meteo = pd.read_csv("df_gps_weather.csv")
test_meteo


,city,lat,lon,jour_+x,temperature_ressentie,probabilite_de_pluie,meteo_principale,score
0,Mont Saint Michel,48.635954,-1.511460,"[1, 2, 3, 4, 5, 6, 7]","[18, 19, 17, 17, 20, 17, 18]","[85, 98, 23, 0, 0, 100, 75]","['Rain', 'Rain', 'Rain', 'Clouds', 'Clouds', '...",12
1,St Malo,48.649518,-2.026041,"[1, 2, 3, 4, 5, 6, 7]","[19, 18, 17, 16, 19, 18, 18]","[99, 88, 15, 9, 0, 100, 65]","['Rain', 'Clouds', 'Clouds', 'Clouds', 'Clouds...",12
2,Bayeux,49.276462,-0.702474,"[1, 2, 3, 4, 5, 6, 7]","[19, 16, 19, 17, 20, 13, 20]","[90, 93, 73, 37, 0, 100, 69]","['Rain', 'Rain', 'Rain', 'Rain', 'Clouds', 'Ra...",11
3,Le Havre,49.493898,0.107973,"[1, 2, 3, 4, 5, 6, 7]","[20, 14, 18, 16, 17, 13, 17]","[74, 100, 25, 18, 0, 98, 64]","['Rain', 'Rain', 'Rain', 'Clouds', 'Clouds', '...",11
4,Rouen,49.440459,1.093966,"[1, 2, 3, 4, 5, 6, 7]","[20, 15, 19, 18, 21, 18, 19]","[97, 100, 30, 61, 0, 42, 62]","['Rain', 'Rain', 'Clouds', 'Rain', 'Clouds', '...",12
5,Paris,48.858890,2.320041,"[1, 2, 3, 4, 5, 6, 7]","[23, 17, 19, 19, 22, 24, 19]","[65, 100, 66, 72, 7, 0, 63]","['Rain', 'Rain', 'Rain', 'Rain', 'Clouds', 'Cl...",15
6,Amiens,49.894171,2.295695,"[1, 2, 3, 4, 5, 6, 7]","[22, 16, 19, 20, 21, 21, 19]","[86, 100, 4, 3, 0, 5, 89]","['Rain', 'Rain', 'Clouds', 'Clouds', 'Clouds',...",15
7,Lille,50.636565,3.063528,"[1, 2, 3, 4, 5, 6, 7]","[22, 19, 20, 21, 21, 22, 19]","[98, 100, 8, 0, 0, 81, 70]","['Rain', 'Rain', 'Clouds', 'Clouds', 'Clear', ...",15
8,Strasbourg,48.584614,7.750713,"[1, 2, 3, 4, 5, 6, 7]","[20, 25, 22, 20, 20, 26, 19]","[100, 92, 100, 100, 53, 0, 36]","['Rain', 'Rain', 'Rain', 'Rain', 'Rain', 'Clou...",14
9,Chateau du Haut Koenigsbourg,48.249523,7.345492,"[1, 2, 3, 4, 5, 6, 7]","[19, 22, 20, 16, 17, 22, 15]","[100, 100, 87, 100, 67, 0, 24]","['Rain', 'Rain', 'Rain', 'Rain', 'Rain', 'Clea...",11


## Create your data lake using S3

In [6]:
# Let's use Amazon S3
s3 = boto3.resource('s3')

In [7]:
# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)

alexlasnier-bucket


In [8]:
bucket = s3.Bucket('alexlasnier-bucket')

In [9]:
'''bucket.upload_file('df_gps_weather.csv', 'meteo.csv')'''

In [14]:
for object_summary in bucket.objects.filter():
    print(object_summary.key)

meteo.csv


## ETL